<a href="https://colab.research.google.com/github/ujanor/NLP/blob/master/5gramtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip3 install google_ngram_api

In [0]:
#from google_ngram_api.Downloader import Downloader

#Creating the object
#downloader = Downloader()

#Calling the functions using the object created
#downloader.download_full_csv('eng','1','a')

In [0]:
import pandas as pd
import io
header_list = ['word', ' year', ' match_count', ' volume_count']
df = pd.read_csv('/content/drive/''My Drive''/''Colab Notebooks''/googlebooks-eng-all-5gram-20120701-_NOUN_',
                 sep='\t',names=header_list)
df

,word,year,match_count,volume_count
0,_NOUN_ _._ _ADJ_ _NUM_ _VERB_,1505,1,1
1,_NOUN_ _._ _ADJ_ _NUM_ _VERB_,1579,3,2
2,_NOUN_ _._ _ADJ_ _NUM_ _VERB_,1587,2,2
3,_NOUN_ _._ _ADJ_ _NUM_ _VERB_,1598,1,1
4,_NOUN_ _._ _ADJ_ _NUM_ _VERB_,1600,2,1
...,...,...,...,...
6716022,_NOUN_ __ _DET_ _ADJ_ _._,2004,5163,3583
6716023,_NOUN_ __ _DET_ _ADJ_ _._,2005,4691,3477
6716024,_NOUN_ __ _DET_ _ADJ_ _._,2006,4957,3598
6716025,_NOUN_ __ _DET_ _ADJ_ _._,2007,5251,3632


In [0]:
df2 = df.head(500000)

In [0]:
print(df2.columns.tolist())

['word', ' year', ' match_count', ' volume_count']


In [0]:
df2['yearBucket'] = df2[' year']-df2[' year'].mod(20) 
df2['avgcount']=df2[' match_count'].div(df2[' volume_count'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
X = df2.drop('yearBucket', axis=1)

y = df2['yearBucket']

In [0]:
x= df2.yearBucket.unique().size

In [0]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
import numpy as np
import re

In [0]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

y = label_encoder.fit_transform(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [0]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:
def preprocess_text(sen):

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [0]:
X1_train = []
sentences = list(X_train["word"])
for sen in sentences:
    X1_train.append(preprocess_text(sen))

In [0]:
X1_test = []
sentences = list(X_test["word"])
for sen in sentences:
    X1_test.append(preprocess_text(sen))

In [0]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X1_train)

X1_train = tokenizer.texts_to_sequences(X1_train)
X1_test = tokenizer.texts_to_sequences(X1_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X1_train = pad_sequences(X1_train, padding='post', maxlen=maxlen)
X1_test = pad_sequences(X1_test, padding='post', maxlen=maxlen)

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/content/drive/''My Drive''/''Colab Notebooks''/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions

glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [0]:
X2_train = X_train[['avgcount']].values
X2_test = X_test[['avgcount']].values

In [0]:
input_1 = Input(shape=(maxlen,))

input_2 = Input(shape=(1,))

In [0]:
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(input_1)
LSTM_Layer_1 = LSTM(128)(embedding_layer)

In [0]:
dense_layer_1 = Dense(128, activation='relu')(input_2)
dense_layer_2 = Dense(128, activation='relu')(dense_layer_1)

In [0]:
concat_layer = Concatenate()([LSTM_Layer_1, dense_layer_2])
dense_layer_3 = Dense(32, activation='relu')(concat_layer)
output = Dense(x, activation='relu')(dense_layer_3)
model = Model(inputs=[input_1, input_2], outputs=output)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 100)     1300        input_3[0][0]                    
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 128)          256         input_4[0][0]                    
____________________________________________________________________________________________

In [0]:
history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.2)

Train on 320000 samples, validate on 80000 samples
Epoch 1/5
  6912/320000 [..............................] - ETA: 14:19 - loss: 5.4817 - acc: 0.0530

KeyboardInterrupt: ignored

In [0]:
score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])